In [233]:
import validation_step
import importlib
importlib.reload(validation_step)
from validation_step import *
from etl_eda import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error


In [234]:
#Loading of datasets
data = load('data.csv')
categories = load('item_categories.csv')
items = load('items.csv')
shops = load('shops.csv')
test = load('./competitive-data-science-predict-future-sales/test.csv')

In [235]:
#Transformation of test set
merge_params = [[items, 'item_id'], [categories, 'item_category_id'], [shops, 'shop_id']]

for df_merge, column in merge_params:
        test = test.merge(df_merge, on = f'{column}', how = 'left')

test['date_block_num'] = 34
test['year'] = 2015
test['month'] = 11

columns = ['item_category_id', 'main_category_id', 'minor_category_id',
       'date_block_num', 'item_id', 'month',
       'year', 'shop_id', 'city_id']
test = test.loc[:, columns]

In [236]:
#Merge Train and Test sets
data = pd.concat([data, test], ignore_index=True, sort=False, keys=columns)

C:\Users\User\AppData\Local\Temp\ipykernel_77908\3268138665.py:2: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  data = pd.concat([data, test], ignore_index=True, sort=False, keys=columns)


In [237]:
#Pipeline with feature enfineering, column trasformation, obtaining a complete data set, validation
categorical_features = ['item_category_id', 'main_category_id', 'minor_category_id', 'shop_id']
log_transform_features = ['item_cnt_month', 'item_price', 'revenue']

lags = [1, 2, 3, 6, 12]
date_block_num = 34

#Pipeline for feature engineering(revenue, shop_history, minor_catregory_history, lags) and obtaining complete data on all item&shop pairs
pipeline_1 = pipeline_1(date_block_num, lags)
pipeline_1.fit(data)
data = pipeline_1.transform(data)

#Pipeline for log transformation and encoding of categorical features
pipeline_2 = pipeline_2(log_transform_features, categorical_features)
pipeline_2.fit(data)
transformed_data = pipeline_2.transform(data)

#Merging results of 2 pipelines
transformed_columns = ['item_cnt_month_log', 'item_price_log', 'revenue_log', 'item_category_id', 'main_category_id', 'minor_category_id', 'shop_id']
transformed_data = pipeline.transform(data)
data[transformed_columns] = transformed_data

#Save in dataframe needed columns
columns = ['item_category_id', 'main_category_id', 'minor_category_id',
       'date_block_num', 'item_id', 'month',
       'year', 'shop_id', 'city_id', 'shop_history',
       'minor_category_history', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_cnt_month_lag_6',
       'item_cnt_month_lag_12', 'item_cnt_month_log', 'item_price_log', 'revenue_log']
data = data.loc[:, columns]

In [240]:
#Data Validation
validation = Validator()
try:
    validated_data = validation.fit_transform(data)
    print("Validation successful, data is valid.")
except ValueError as e:
    print(f"Validation error: {e}")
except TypeError as e:
    print(f"Type error: {e}")

Validation successful, data is valid.


In [46]:
X_test = transformed_data.drop('item_cnt_month', axis = 1).loc[transformed_data.date_block_num == 34]
X_train = transformed_data.drop('item_cnt_month', axis = 1).loc[transformed_data.date_block_num != 34]

In [4]:
model = DummyRegressor(strategy= 'mean')

Train/val split + check if we have enough data

In [5]:
tss = TimeSeriesSplit(n_splits = 3)
n = 0

for train_index, validation_index in tss.split(data.loc[data.date_block_num != 34]):
    train_data, validation_data = data.iloc[train_index], data.iloc[validation_index]
    X_train = train_data.drop('item_cnt_month_log', axis = 1)
    y_train = train_data['item_cnt_month_log']
    X_val = validation_data.drop('item_cnt_month_log', axis = 1)
    y_val = validation_data['item_cnt_month_log']

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_pred, y_val)
    n += 1
    print(f'MSE for fold №{n}: {mse}')

MSE for fold №1: 0.035554202977984124
MSE for fold №2: 0.026481845637343997
MSE for fold №3: 0.02788435463394971


The obtained MSE we could use in future as baselines, to ensure that we do not have data leakage